In [31]:
import warnings

warnings.filterwarnings("ignore")  # Suprime warnings

import pandas as pd
import numpy as np
import pickle
import os

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [11]:
# Ruta de los outputs
output_folder = "../../data/outputs/3_eda"

# Cargar el DataFrame preprocesado que incluye el texto concatenado y la variable objetivo
texto_medicamentos_df = pd.read_csv(
    os.path.join(output_folder, "texto_concatenado_medicamentos.csv")
)
print("Columnas:", texto_medicamentos_df.columns.tolist())

# Aseguramos que la variable objetivo esté presente (por ejemplo, "descripcion_nivel_anatomico")
assert (
    "descripcion_nivel_anatomico" in texto_medicamentos_df.columns
), "No se encontró la variable objetivo."

Columnas: ['medicamento', 'descripcion_nivel_anatomico', 'descripcion_nivel_2_subgrupo_terapeutico', 'descripcion_nivel_3_subgrupo_terapeutico_farmacologico', 'descripcion_nivel_4_subgrupo_terapeutico_farmacologico_quimico', 'descripcion_nivel_5_principio_activo', 'texto_completo']


In [42]:
# Cargar el vectorizador TF-IDF y la matriz vectorizada
with open(os.path.join(output_folder, "tfidf_vectorizer.pkl"), "rb") as f:
    tfidf_vectorizer = pickle.load(f)

with open(os.path.join(output_folder, "tfidf_matrix.pkl"), "rb") as f:
    tfidf_matrix = pickle.load(f)

# Definir features (X) y variable objetivo (y)
X = tfidf_matrix  # Matriz TF-IDF
# Queremos predecir el nivel anatomico, es decir, sistema nervioso, digestivo, etc.
y = texto_medicamentos_df["descripcion_nivel_anatomico"]
# Rellenar valores nulos
y = y.fillna("Desconocido")

# Convertir categorías a valores numéricos
le = LabelEncoder()
y = le.fit_transform(y) # 1,2,3,4,5, ...
# categorias_originales = le.inverse_transform(y) # si queremos revertir la codificación


In [43]:
# Dividir en entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [45]:
#####################################
# 1. Optimización de hiperparámetros #
#####################################

In [46]:
# --- Regresión Logística ---
print("### Grid Search: Regresión Logística ###")

# Definir el grid de hiperparámetros. Por ejemplo, se puede optimizar el parámetro C
param_grid_lr = {
    "C": [0.01, 0.1, 1, 10, 100],
    "penalty": ["l2"],  # Para 'l1' se necesita solver 'liblinear'
    "solver": ["lbfgs"],  # lbfgs soporta multiclass
    "max_iter": [1000],
}

### Grid Search: Regresión Logística ###


In [47]:
lr = LogisticRegression(random_state=42)
grid_lr = GridSearchCV(lr, param_grid_lr, cv=5, scoring="accuracy", n_jobs=-1)
grid_lr.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print("Mejores hiperparámetros (LogReg):", grid_lr.best_params_)
print("Mejor CV Accuracy (LogReg): {:.2f}".format(grid_lr.best_score_))

Mejores hiperparámetros (LogReg): {'C': 10, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
Mejor CV Accuracy (LogReg): 0.85


In [ ]:
# Evaluación en el conjunto de entrenamiento y test
best_lr = grid_lr.best_estimator_
y_train_pred_lr = best_lr.predict(X_train)
y_test_pred_lr = best_lr.predict(X_test)

print(
    "Regresión Logística - Train Accuracy: {:.2f}".format(
        accuracy_score(y_train, y_train_pred_lr)
    )
)
print(
    "Regresión Logística - Test Accuracy: {:.2f}".format(
        accuracy_score(y_test, y_test_pred_lr)
    )
)
print(
    "\nReporte de Clasificación - Regresión Logística (Test):\n",
    classification_report(y_test, y_test_pred_lr, zero_division=0),
)

Regresión Logística - Train Accuracy: 0.97
Regresión Logística - Test Accuracy: 0.90

Reporte de Clasificación - Regresión Logística (Test):
                              precision    recall  f1-score   support

                 ANALGESICS       0.67      1.00      0.80         2
ANTIVIRALS FOR SYSTEMIC USE       1.00      1.00      1.00         3
     LIPID MODIFYING AGENTS       1.00      1.00      1.00         2
              PSYCHOLEPTICS       1.00      1.00      1.00         2
                UROLOGICALS       0.00      0.00      0.00         1

                   accuracy                           0.90        10
                  macro avg       0.73      0.80      0.76        10
               weighted avg       0.83      0.90      0.86        10



In [ ]:
# --- RandomForest ---
print("\n### Grid Search: RandomForestClassifier ###")

param_grid_rf = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
}


### Grid Search: RandomForestClassifier ###


In [ ]:
rf = RandomForestClassifier(random_state=42)
grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring="accuracy", n_jobs=-1)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [12]:
print("Mejores hiperparámetros (RF):", grid_rf.best_params_)
print("Mejor CV Accuracy (RF): {:.2f}".format(grid_rf.best_score_))

Mejores hiperparámetros (RF): {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Mejor CV Accuracy (RF): 0.85


In [ ]:
# Evaluación en el conjunto de entrenamiento y test
best_rf = grid_rf.best_estimator_
y_train_pred_rf = best_rf.predict(X_train)
y_test_pred_rf = best_rf.predict(X_test)

print(
    "RandomForest - Train Accuracy: {:.2f}".format(
        accuracy_score(y_train, y_train_pred_rf)
    )
)
print(
    "RandomForest - Test Accuracy: {:.2f}".format(
        accuracy_score(y_test, y_test_pred_rf)
    )
)
print(
    "\nReporte de Clasificación - RandomForest (Test):\n",
    classification_report(y_test, y_test_pred_rf, zero_division=0),
)

RandomForest - Train Accuracy: 1.00
RandomForest - Test Accuracy: 0.90

Reporte de Clasificación - RandomForest (Test):
                              precision    recall  f1-score   support

                 ANALGESICS       0.67      1.00      0.80         2
ANTIVIRALS FOR SYSTEMIC USE       1.00      1.00      1.00         3
     LIPID MODIFYING AGENTS       1.00      1.00      1.00         2
              PSYCHOLEPTICS       1.00      1.00      1.00         2
                UROLOGICALS       0.00      0.00      0.00         1

                   accuracy                           0.90        10
                  macro avg       0.73      0.80      0.76        10
               weighted avg       0.83      0.90      0.86        10



In [14]:
############################################
# 2. Interpretabilidad: Variables Explicativas #
############################################

# --- Para Regresión Logística ---
# Extraer coeficientes: best_lr.coef_ tiene forma (n_clases, n_features)
feature_names = tfidf_vectorizer.get_feature_names_out()
coef = best_lr.coef_

# Por ejemplo, para cada clase mostramos las 10 características con coeficiente más alto
print("\nTop características por clase (Regresión Logística):")
for idx, class_label in enumerate(best_lr.classes_):
    coef_class = coef[idx]
    # Ordenar índices de mayor a menor
    top10_idx = np.argsort(coef_class)[-10:]
    top_features = feature_names[top10_idx]
    top_coef = coef_class[top10_idx]
    print(f"\nClase: {class_label}")
    for feat, coef_val in zip(top_features[::-1], top_coef[::-1]):
        print(f"{feat}: {coef_val:.4f}")


# --- Para RandomForest ---
# Extraer feature_importances_
importances = best_rf.feature_importances_
indices_rf = np.argsort(importances)[-10:]
print("\nTop características (RandomForest):")
for idx in indices_rf[::-1]:
    print(f"{feature_names[idx]}: {importances[idx]:.4f}")


Top características por clase (Regresión Logística):

Clase: AGENTS ACTING ON THE RENIN-ANGIOTENSIN SYSTEM
enalapril: 2.1722
eca: 1.8020
inhibidores: 0.5709
renal: 0.4445
función renal: 0.3388
arterial: 0.3136
insuficiencia: 0.2796
pacientes: 0.2626
hipotensión: 0.2570
microgramos: 0.2435

Clase: ANALGESICS
fentanilo: 2.3316
paracetamol: 2.0942
abfentiq: 1.7492
abattra: 0.9452
opioides: 0.7311
dosis: 0.6388
transdérmico: 0.6228
parche: 0.5790
fentanilo transdérmico: 0.5768
dolor: 0.4825

Clase: ANTIDIARRHEALS, INTESTINAL ANTIINFLAMMATORY/ANTIINFECTIVE AGENTS
loperamida: 3.0203
comprimidos: 0.2367
diarrea: 0.1815
sobredosis: 0.1560
aguda: 0.1335
estreñimiento: 0.1179
principal: 0.1168
snc: 0.1142
clínicos: 0.1141
raras: 0.1066

Clase: ANTIMYCOTICS FOR SYSTEMIC USE
complejo: 2.5995
frecuente: 0.9941
renal: 0.4908
mg kg: 0.4208
pacientes: 0.3416
kg: 0.2943
tratamiento: 0.2881
reacciones: 0.2840
días: 0.2370
función: 0.2283

Clase: ANTITHROMBOTIC AGENTS
ácido acetilsalicílico: 1.4106
ácid

In [15]:
############################################
# 3. Prueba manual de un medicamento       #
############################################


# Seleccionar manualmente un ejemplo del DataFrame (por ejemplo, la primera fila)
manual_sample = texto_medicamentos_df.iloc[0]
sample_text = manual_sample["texto_completo"]

# Transformar el texto a la representación TF-IDF (usando el vectorizador cargado)
sample_vector = tfidf_vectorizer.transform([sample_text])

# Predecir con ambos modelos (y obtener probabilidades)
lr_proba = best_lr.predict_proba(sample_vector)[0]
rf_proba = best_rf.predict_proba(sample_vector)[0]

print("\n--- Predicción manual ---")
print("Medicamento:", manual_sample["medicamento"])
print("Texto (truncado):", sample_text[:200], "...")
print("\nPredicción Regresión Logística:")
for label, prob in zip(best_lr.classes_, lr_proba):
    print(f"{label}: {prob:.2f}")

print("\nPredicción RandomForest:")
for label, prob in zip(best_rf.classes_, rf_proba):
    print(f"{label}: {prob:.2f}")

# También podemos ver la predicción final
print("\nPredicción final (LogReg):", best_lr.predict(sample_vector)[0])
print("Predicción final (RF):", best_rf.predict(sample_vector)[0])


--- Predicción manual ---
Medicamento: A.A.S._100_mg_COMPRIMIDOS.txt
Texto (truncado): en base a su efecto antiagregante plaquetario está indicado en la profilaxis de infarto de miocardio o reinfarto de miocardio en pacientes con angina de pecho inestable y para prevenir la recurrencia  ...

Predicción Regresión Logística:
AGENTS ACTING ON THE RENIN-ANGIOTENSIN SYSTEM: 0.03
ANALGESICS: 0.45
ANTIDIARRHEALS, INTESTINAL ANTIINFLAMMATORY/ANTIINFECTIVE AGENTS: 0.02
ANTIMYCOTICS FOR SYSTEMIC USE: 0.03
ANTITHROMBOTIC AGENTS: 0.36
ANTIVIRALS FOR SYSTEMIC USE: 0.04
CORTICOSTEROIDS, DERMATOLOGICAL PREPARATIONS: 0.02
LIPID MODIFYING AGENTS: 0.03
PSYCHOLEPTICS: 0.03

Predicción RandomForest:
AGENTS ACTING ON THE RENIN-ANGIOTENSIN SYSTEM: 0.00
ANALGESICS: 0.32
ANTIDIARRHEALS, INTESTINAL ANTIINFLAMMATORY/ANTIINFECTIVE AGENTS: 0.02
ANTIMYCOTICS FOR SYSTEMIC USE: 0.00
ANTITHROMBOTIC AGENTS: 0.54
ANTIVIRALS FOR SYSTEMIC USE: 0.02
CORTICOSTEROIDS, DERMATOLOGICAL PREPARATIONS: 0.02
LIPID MODIFYING AGENT